In [1]:
import pyspark
import re
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer,RegexTokenizer
from pyspark.sql.functions import col,udf
from pyspark.sql.types import IntegerType
spark= SparkSession.builder.appName('proyecto3').getOrCreate()

In [2]:
df1= spark.read.csv('/FileStore/tables/articles1.csv',inferSchema=True,header=True)

In [3]:
df1.show()

+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
_c0| id| title| publication| author| date| year|month| url| content|
+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
 0|17283|House Republicans...|New York Times| Carl Hulse|2016-12-31|2016.0| 12.0|null|WASHINGTON — C...|
 1|17284|Rift Between Offi...|New York Times|Benjamin Mueller ...|2017-06-19|2017.0| 6.0|null|After the bullet ...|
 2|17285|Tyrus Wong, ‘Bamb...|New York Times| Margalit Fox|2017-01-06|2017.0| 1.0|null|When Walt Disney’...|
 3|17286|Among Deaths in 2...|New York Times| William McDonald|2017-04-10|2017.0| 4.0|null|Death may be the ...|
 4|17287|Kim Jong-un Says ...|New York Times| Choe Sang-Hun|2017-01-02|2017.0| 1.0|null|SEOUL, South Kore...|
 5|17288|Sick With a Cold,...|New York Times| Sewell Chan|2017-01-02|2017.0| 1.0|null|LONDON — Queen...|
 6|17289|Taiwan’s Presiden...|New York Times| Javier C. Hernández|2017-01-02|2017.0| 1.0|null|BEIJING — Pres...|
 7|17290|After ‘The Bigges...|New York Times| Gina Kolata|2017-02-08|2017.0| 2.0|null|Danny Cahill stoo...|
 8|17291|First, a Mixtape....|New York Times| Katherine Rosman|2016-12-31|2016.0| 12.0|null|Just how is Hil...|
 9|17292|Calling on Angels...|New York Times| Andy Newman|2016-12-31|2016.0| 12.0|null|Angels are everyw...|
 10|17293|Weak Federal Powe...|New York Times| Justin Gillis|2017-01-03|2017.0| 1.0|null|With Donald J. Tr...|
 11|17294|Can Carbon Captur...|New York Times| John Schwartz|2017-01-05|2017.0| 1.0|null|THOMPSONS, Tex. ...|
 12|17295|Mar-a-Lago, the F...|New York Times| Maggie Haberman|2017-01-02|2017.0| 1.0|null|WEST PALM BEACH, ...|
 13|17296|How to form healt...|New York Times| Charles Duhigg|2017-01-02|2017.0| 1.0|null|This article is p...|
 14|17297|Turning Your Vaca...|New York Times|Stephanie Rosenbloom|2017-04-14|2017.0| 4.0|null|It’s the season f...|
 15|17298|As Second Avenue ...|New York Times| Emma G. Fitzsimmons|2017-01-02|2017.0| 1.0|null|Finally. The Seco...|
 16|17300|Dylann Roof Himse...|New York Times|Kevin Sack and Al...|2017-01-02|2017.0| 1.0|null| pages into the ...|
 17|17301|Modi’s Cash Ban B...|New York Times| Geeta Anand|2017-01-02|2017.0| 1.0|null|MUMBAI, India — ...|
 18|17302|Suicide Bombing i...|New York Times|The Associated Press|2017-01-03|2017.0| 1.0|null|BAGHDAD — A su...|
 19|17303|Fecal Pollution T...|New York Times| Brett Cole|2017-01-03|2017.0| 1.0|null|SYDNEY, Australia...|
+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
only showing top 20 rows

In [4]:
df2= spark.read.csv('/FileStore/tables/articles2.csv',inferSchema=True,header=True)

In [5]:
df3= spark.read.csv('/FileStore/tables/articles3.csv',inferSchema=True,header=True)

In [6]:
dfx=df1.union(df2)

In [7]:
dft=dfx.union(df3)

In [8]:
dft.show()

+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
_c0| id| title| publication| author| date| year|month| url| content|
+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
 0|17283|House Republicans...|New York Times| Carl Hulse|2016-12-31|2016.0| 12.0|null|WASHINGTON — C...|
 1|17284|Rift Between Offi...|New York Times|Benjamin Mueller ...|2017-06-19|2017.0| 6.0|null|After the bullet ...|
 2|17285|Tyrus Wong, ‘Bamb...|New York Times| Margalit Fox|2017-01-06|2017.0| 1.0|null|When Walt Disney’...|
 3|17286|Among Deaths in 2...|New York Times| William McDonald|2017-04-10|2017.0| 4.0|null|Death may be the ...|
 4|17287|Kim Jong-un Says ...|New York Times| Choe Sang-Hun|2017-01-02|2017.0| 1.0|null|SEOUL, South Kore...|
 5|17288|Sick With a Cold,...|New York Times| Sewell Chan|2017-01-02|2017.0| 1.0|null|LONDON — Queen...|
 6|17289|Taiwan’s Presiden...|New York Times| Javier C. Hernández|2017-01-02|2017.0| 1.0|null|BEIJING — Pres...|
 7|17290|After ‘The Bigges...|New York Times| Gina Kolata|2017-02-08|2017.0| 2.0|null|Danny Cahill stoo...|
 8|17291|First, a Mixtape....|New York Times| Katherine Rosman|2016-12-31|2016.0| 12.0|null|Just how is Hil...|
 9|17292|Calling on Angels...|New York Times| Andy Newman|2016-12-31|2016.0| 12.0|null|Angels are everyw...|
 10|17293|Weak Federal Powe...|New York Times| Justin Gillis|2017-01-03|2017.0| 1.0|null|With Donald J. Tr...|
 11|17294|Can Carbon Captur...|New York Times| John Schwartz|2017-01-05|2017.0| 1.0|null|THOMPSONS, Tex. ...|
 12|17295|Mar-a-Lago, the F...|New York Times| Maggie Haberman|2017-01-02|2017.0| 1.0|null|WEST PALM BEACH, ...|
 13|17296|How to form healt...|New York Times| Charles Duhigg|2017-01-02|2017.0| 1.0|null|This article is p...|
 14|17297|Turning Your Vaca...|New York Times|Stephanie Rosenbloom|2017-04-14|2017.0| 4.0|null|It’s the season f...|
 15|17298|As Second Avenue ...|New York Times| Emma G. Fitzsimmons|2017-01-02|2017.0| 1.0|null|Finally. The Seco...|
 16|17300|Dylann Roof Himse...|New York Times|Kevin Sack and Al...|2017-01-02|2017.0| 1.0|null| pages into the ...|
 17|17301|Modi’s Cash Ban B...|New York Times| Geeta Anand|2017-01-02|2017.0| 1.0|null|MUMBAI, India — ...|
 18|17302|Suicide Bombing i...|New York Times|The Associated Press|2017-01-03|2017.0| 1.0|null|BAGHDAD — A su...|
 19|17303|Fecal Pollution T...|New York Times| Brett Cole|2017-01-03|2017.0| 1.0|null|SYDNEY, Australia...|
+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
only showing top 20 rows

In [9]:
df1.show()

+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
_c0| id| title| publication| author| date| year|month| url| content|
+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
 0|17283|House Republicans...|New York Times| Carl Hulse|2016-12-31|2016.0| 12.0|null|WASHINGTON — C...|
 1|17284|Rift Between Offi...|New York Times|Benjamin Mueller ...|2017-06-19|2017.0| 6.0|null|After the bullet ...|
 2|17285|Tyrus Wong, ‘Bamb...|New York Times| Margalit Fox|2017-01-06|2017.0| 1.0|null|When Walt Disney’...|
 3|17286|Among Deaths in 2...|New York Times| William McDonald|2017-04-10|2017.0| 4.0|null|Death may be the ...|
 4|17287|Kim Jong-un Says ...|New York Times| Choe Sang-Hun|2017-01-02|2017.0| 1.0|null|SEOUL, South Kore...|
 5|17288|Sick With a Cold,...|New York Times| Sewell Chan|2017-01-02|2017.0| 1.0|null|LONDON — Queen...|
 6|17289|Taiwan’s Presiden...|New York Times| Javier C. Hernández|2017-01-02|2017.0| 1.0|null|BEIJING — Pres...|
 7|17290|After ‘The Bigges...|New York Times| Gina Kolata|2017-02-08|2017.0| 2.0|null|Danny Cahill stoo...|
 8|17291|First, a Mixtape....|New York Times| Katherine Rosman|2016-12-31|2016.0| 12.0|null|Just how is Hil...|
 9|17292|Calling on Angels...|New York Times| Andy Newman|2016-12-31|2016.0| 12.0|null|Angels are everyw...|
 10|17293|Weak Federal Powe...|New York Times| Justin Gillis|2017-01-03|2017.0| 1.0|null|With Donald J. Tr...|
 11|17294|Can Carbon Captur...|New York Times| John Schwartz|2017-01-05|2017.0| 1.0|null|THOMPSONS, Tex. ...|
 12|17295|Mar-a-Lago, the F...|New York Times| Maggie Haberman|2017-01-02|2017.0| 1.0|null|WEST PALM BEACH, ...|
 13|17296|How to form healt...|New York Times| Charles Duhigg|2017-01-02|2017.0| 1.0|null|This article is p...|
 14|17297|Turning Your Vaca...|New York Times|Stephanie Rosenbloom|2017-04-14|2017.0| 4.0|null|It’s the season f...|
 15|17298|As Second Avenue ...|New York Times| Emma G. Fitzsimmons|2017-01-02|2017.0| 1.0|null|Finally. The Seco...|
 16|17300|Dylann Roof Himse...|New York Times|Kevin Sack and Al...|2017-01-02|2017.0| 1.0|null| pages into the ...|
 17|17301|Modi’s Cash Ban B...|New York Times| Geeta Anand|2017-01-02|2017.0| 1.0|null|MUMBAI, India — ...|
 18|17302|Suicide Bombing i...|New York Times|The Associated Press|2017-01-03|2017.0| 1.0|null|BAGHDAD — A su...|
 19|17303|Fecal Pollution T...|New York Times| Brett Cole|2017-01-03|2017.0| 1.0|null|SYDNEY, Australia...|
+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
only showing top 20 rows

In [10]:
df2.show()

+-----+-----+--------------------+-----------+--------------------+----------+------+-----+----+--------------------+
 _c0| id| title|publication| author| date| year|month| url| content|
+-----+-----+--------------------+-----------+--------------------+----------+------+-----+----+--------------------+
53293|73471|Patriots Day Is B...| Atlantic| David Sims|2017-01-11|2017.0| 1.0|null|Patriots Day, Pet...|
53294|73472|A Break in the Se...| Atlantic| Ed Yong|2017-01-11|2017.0| 1.0|null|In Norse mytholog...|
53295|73474|Obama’s Ingenious...| Atlantic| Spencer Kornhaber|2017-01-11|2017.0| 1.0|null|“If our democracy...|
53296|73475|Donald Trump Meet...| Atlantic| David A. Graham|2017-01-11|2017.0| 1.0|null|Updated on Januar...|
53297|73476|Trump: ’I Think’ ...| Atlantic| Kaveh Waddell|2017-01-11|2017.0| 1.0|null|Updated at 12:25 ...|
53298|73477|Seth Meyers Quest...| Atlantic| Megan Garber|2017-01-11|2017.0| 1.0|null|Here was one of t...|
53299|73478|Obama Frames His ...| Atlantic| Gillian B. White|2017-01-11|2017.0| 1.0|null|In Barack Obama’s...|
53300|73479|The Trump Adminis...| Atlantic| Jeremy Venook|2017-01-11|2017.0| 1.0|null|With confirmation...|
53301|73484|The Longstanding ...| Atlantic| Alia Wong|2017-01-12|2017.0| 1.0|null|Havasu Canyon is ...|
53302|73485|The Atlantic Dail...| Atlantic|Rosa Inocencio Smith|2017-01-12|2017.0| 1.0|null| This arti...|
53303|73486|A Pledge for More...| Atlantic| Andrew J. Bacevich|2017-01-12|2017.0| 1.0|null|That retired Gene...|
53304|73487|The Atlantic Pol...| Atlantic| Elaine Godfrey|2017-01-12|2017.0| 1.0|null| This a...|
53305|73488|The Contradiction...| Atlantic| Alana Semuels|2017-01-12|2017.0| 1.0|null|When he was on th...|
53306|73489|A Sanctions Skept...| Atlantic| Siddhartha Mahanta|2017-01-12|2017.0| 1.0|null|Rex Tillerson, Do...|
53307|73490|Baltimore Police ...| Atlantic| David A. Graham|2017-01-12|2017.0| 1.0|null|The U. S. Departm...|
53308|73491|About That Young ...| Atlantic| Sophie Gilbert|2017-01-12|2017.0| 1.0|null|Imagine, if you c...|
53309|73492|The DeVos Hearing...| Atlantic| Emily DeRuy|2017-01-12|2017.0| 1.0|null|The Trump adminis...|
53310|73493|Donald Trump Says...| Atlantic| Derek Thompson|2017-01-12|2017.0| 1.0|null|Donald Trump’s fi...|
53311|73494|A ‘One-Stop Shop’...| Atlantic| Rosie Gray|2017-01-12|2017.0| 1.0|null|Richard Spencer, ...|
53312|73495|Rich Students Go ...| Atlantic| Mikhail Zinshteyn|2017-01-12|2017.0| 1.0|null|More Americans ar...|
+-----+-----+--------------------+-----------+--------------------+----------+------+-----+----+--------------------+
only showing top 20 rows

In [11]:
df3.show()

+------+------+--------------------+-----------+-----------------+----------+------+-----+--------------------+--------------------+
 _c0| id| title|publication| author| date| year|month| url| content|
+------+------+--------------------+-----------+-----------------+----------+------+-----+--------------------+--------------------+
103459|151908|Alton Sterling’s ...| Guardian| Jessica Glenza|2016-07-13|2016.0| 7.0|https://www.thegu...|The son of a Loui...|
103460|151909|Shakespeare’s fir...| Guardian| null|2016-05-25|2016.0| 5.0|https://www.thegu...|Copies of William...|
103461|151910|My grandmother’s ...| Guardian| Robert Pendry|2016-10-31|2016.0| 10.0|https://www.thegu...|Debt: $20, 000, S...|
103462|151911|I feared my life ...| Guardian| Bradford Frost|2016-11-26|2016.0| 11.0|https://www.thegu...|It was late. I wa...|
103463|151912|Texas man serving...| Guardian| null|2016-08-20|2016.0| 8.0|https://www.thegu...|A central Texas m...|
103464|151914|My dad’s Reagan p...| Guardian|Steven W Thrasher|2016-11-28|2016.0| 11.0|https://www.thegu...|I have been battl...|
103465|151915|Flatmates of gay ...| Guardian| Patrick Kingsley|2016-08-07|2016.0| 8.0|https://www.thegu...|Three flatmates o...|
103466|151916|Jaffas and darede...| Guardian|Eleanor Ainge Roy|2016-07-22|2016.0| 7.0|https://www.thegu...|, Most people tak...|
103467|151917|NSA contractor ar...| Guardian| Ewen MacAskill|2016-10-05|2016.0| 10.0|https://www.thegu...|The FBI has arres...|
103468|151918|Donald Trump to d...| Guardian| Ben Jacobs|2016-12-24|2016.0| 12.0|https://www.thegu...|Donald Trump anno...|
103469|151919|Serbian Olympic r...| Guardian| Sean Ingle|2016-08-06|2016.0| 8.0|https://www.thegu...|Most Olympic rowe...|
103470|151920|The strange case ...| Guardian| null|2016-10-17|2016.0| 10.0|https://www.thegu...|Name: Pamela Ande...|
103471|151921|‘Voting for a rac...| Guardian| Sabrina Siddiqui|2016-09-25|2016.0| 9.0|https://www.thegu...|From the moment i...|
103472|151923|Push for ban on d...| Guardian| Oliver Milman|2016-09-10|2016.0| 9.0|https://www.thegu...|A resolution to e...|
103473|151924|Bad moms: why mes...| Guardian| Lara Williams|2016-10-02|2016.0| 10.0|https://www.thegu...|Better Things ope...|
103474|151925|Planned Parenthoo...| Guardian| Molly Redden|2016-06-30|2016.0| 6.0|https://www.thegu...|Efforts are brewi...|
103475|151926|’Trust can be res...| Guardian| Guardian readers|2016-10-29|2016.0| 10.0|https://www.thegu...|Halldora was born...|
103476|151927|Clinton endorsed ...| Guardian| Martin Pengelly|2016-09-24|2016.0| 9.0|https://www.thegu...|As Hillary Clinto...|
103477|151928|Ignore the naysay...| Guardian| Aryeh Neier|2016-09-16|2016.0| 9.0|https://www.thegu...|Barack Obama’s an...|
103478|151930|Golden State Warr...| Guardian| Arielle Retting|2016-05-30|2016.0| 5.0|https://www.thegu...|The Golden State ...|
+------+------+--------------------+-----------+-----------------+----------+------+-----+--------------------+--------------------+
only showing top 20 rows

In [12]:
df=dft.select(dft['id'],dft['title'],dft['content'])

In [13]:
df.show()

+-----+--------------------+--------------------+
 id| title| content|
+-----+--------------------+--------------------+
17283|House Republicans...|WASHINGTON — C...|
17284|Rift Between Offi...|After the bullet ...|
17285|Tyrus Wong, ‘Bamb...|When Walt Disney’...|
17286|Among Deaths in 2...|Death may be the ...|
17287|Kim Jong-un Says ...|SEOUL, South Kore...|
17288|Sick With a Cold,...|LONDON — Queen...|
17289|Taiwan’s Presiden...|BEIJING — Pres...|
17290|After ‘The Bigges...|Danny Cahill stoo...|
17291|First, a Mixtape....|Just how is Hil...|
17292|Calling on Angels...|Angels are everyw...|
17293|Weak Federal Powe...|With Donald J. Tr...|
17294|Can Carbon Captur...|THOMPSONS, Tex. ...|
17295|Mar-a-Lago, the F...|WEST PALM BEACH, ...|
17296|How to form healt...|This article is p...|
17297|Turning Your Vaca...|It’s the season f...|
17298|As Second Avenue ...|Finally. The Seco...|
17300|Dylann Roof Himse...| pages into the ...|
17301|Modi’s Cash Ban B...|MUMBAI, India — ...|
17302|Suicide Bombing i...|BAGHDAD — A su...|
17303|Fecal Pollution T...|SYDNEY, Australia...|
+-----+--------------------+--------------------+
only showing top 20 rows

In [14]:
#Funcion para indexar revisar columnas
#from pyspark.sql.functions import monotonically_increasing_id
#dflast = dft.withColumn("index", monotonically_increasing_id())
#dflast.createOrReplaceTempView("table_df")
#dfobserve= sqlContext.sql("SELECT * FROM table_df ORDER BY index DESC limit 5")
#dfobserve.show()

In [15]:
tokenizer= Tokenizer(inputCol='content', outputCol='words')
regex_tokenizer= RegexTokenizer(inputCol='content', outputCol='words',pattern='[^a-zA-Z0-9]+').setMinTokenLength(2)

In [16]:
count_tokens= udf(lambda words:len(words),IntegerType())

In [17]:
df2=df.fillna("unknown", subset=["content","title"])
tokenized= regex_tokenizer.transform(df2)

In [18]:
tokenized.show()

+-----+--------------------+--------------------+--------------------+
 id| title| content| words|
+-----+--------------------+--------------------+--------------------+
17283|House Republicans...|WASHINGTON — C...|[washington, cong...|
17284|Rift Between Offi...|After the bullet ...|[after, the, bull...|
17285|Tyrus Wong, ‘Bamb...|When Walt Disney’...|[when, walt, disn...|
17286|Among Deaths in 2...|Death may be the ...|[death, may, be, ...|
17287|Kim Jong-un Says ...|SEOUL, South Kore...|[seoul, south, ko...|
17288|Sick With a Cold,...|LONDON — Queen...|[london, queen, e...|
17289|Taiwan’s Presiden...|BEIJING — Pres...|[beijing, preside...|
17290|After ‘The Bigges...|Danny Cahill stoo...|[danny, cahill, s...|
17291|First, a Mixtape....|Just how is Hil...|[just, how, is, h...|
17292|Calling on Angels...|Angels are everyw...|[angels, are, eve...|
17293|Weak Federal Powe...|With Donald J. Tr...|[with, donald, tr...|
17294|Can Carbon Captur...|THOMPSONS, Tex. ...|[thompsons, tex, ...|
17295|Mar-a-Lago, the F...|WEST PALM BEACH, ...|[west, palm, beac...|
17296|How to form healt...|This article is p...|[this, article, i...|
17297|Turning Your Vaca...|It’s the season f...|[it, the, season,...|
17298|As Second Avenue ...|Finally. The Seco...|[finally, the, se...|
17300|Dylann Roof Himse...| pages into the ...|[pages, into, the...|
17301|Modi’s Cash Ban B...|MUMBAI, India — ...|[mumbai, india, i...|
17302|Suicide Bombing i...|BAGHDAD — A su...|[baghdad, suicide...|
17303|Fecal Pollution T...|SYDNEY, Australia...|[sydney, australi...|
+-----+--------------------+--------------------+--------------------+
only showing top 20 rows

In [19]:
from pyspark.ml.feature import StopWordsRemover
remover= StopWordsRemover(inputCol='words', outputCol='filtered')

In [20]:
sinsw=remover.transform(tokenized)

In [21]:
sinsw.show()

+-----+--------------------+--------------------+--------------------+--------------------+
 id| title| content| words| filtered|
+-----+--------------------+--------------------+--------------------+--------------------+
17283|House Republicans...|WASHINGTON — C...|[washington, cong...|[washington, cong...|
17284|Rift Between Offi...|After the bullet ...|[after, the, bull...|[bullet, shells, ...|
17285|Tyrus Wong, ‘Bamb...|When Walt Disney’...|[when, walt, disn...|[walt, disney, ba...|
17286|Among Deaths in 2...|Death may be the ...|[death, may, be, ...|[death, may, grea...|
17287|Kim Jong-un Says ...|SEOUL, South Kore...|[seoul, south, ko...|[seoul, south, ko...|
17288|Sick With a Cold,...|LONDON — Queen...|[london, queen, e...|[london, queen, e...|
17289|Taiwan’s Presiden...|BEIJING — Pres...|[beijing, preside...|[beijing, preside...|
17290|After ‘The Bigges...|Danny Cahill stoo...|[danny, cahill, s...|[danny, cahill, s...|
17291|First, a Mixtape....|Just how is Hil...|[just, how, is, h...|[hillary, kerr, f...|
17292|Calling on Angels...|Angels are everyw...|[angels, are, eve...|[angels, everywhe...|
17293|Weak Federal Powe...|With Donald J. Tr...|[with, donald, tr...|[donald, trump, t...|
17294|Can Carbon Captur...|THOMPSONS, Tex. ...|[thompsons, tex, ...|[thompsons, tex, ...|
17295|Mar-a-Lago, the F...|WEST PALM BEACH, ...|[west, palm, beac...|[west, palm, beac...|
17296|How to form healt...|This article is p...|[this, article, i...|[article, part, s...|
17297|Turning Your Vaca...|It’s the season f...|[it, the, season,...|[season, family, ...|
17298|As Second Avenue ...|Finally. The Seco...|[finally, the, se...|[finally, second,...|
17300|Dylann Roof Himse...| pages into the ...|[pages, into, the...|[pages, journal, ...|
17301|Modi’s Cash Ban B...|MUMBAI, India — ...|[mumbai, india, i...|[mumbai, india, b...|
17302|Suicide Bombing i...|BAGHDAD — A su...|[baghdad, suicide...|[baghdad, suicide...|
17303|Fecal Pollution T...|SYDNEY, Australia...|[sydney, australi...|[sydney, australi...|
+-----+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [22]:
from pyspark.ml.feature import HashingTF,IDF,Tokenizer


In [23]:
hashing_tf= HashingTF(inputCol='filtered',outputCol='rawFeatures')

In [24]:
featurized_data=hashing_tf.transform(sinsw)

In [25]:
featurized_data.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 id| title| content| words| filtered| rawFeatures|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
17283|House Republicans...|WASHINGTON — C...|[washington, cong...|[washington, cong...|(262144,[619,2326...|
17284|Rift Between Offi...|After the bullet ...|[after, the, bull...|[bullet, shells, ...|(262144,[14,343,1...|
17285|Tyrus Wong, ‘Bamb...|When Walt Disney’...|[when, walt, disn...|[walt, disney, ba...|(262144,[14,535,5...|
17286|Among Deaths in 2...|Death may be the ...|[death, may, be, ...|[death, may, grea...|(262144,[75,535,7...|
17287|Kim Jong-un Says ...|SEOUL, South Kore...|[seoul, south, ko...|[seoul, south, ko...|(262144,[2472,328...|
17288|Sick With a Cold,...|LONDON — Queen...|[london, queen, e...|[london, queen, e...|(262144,[2564,538...|
17289|Taiwan’s Presiden...|BEIJING — Pres...|[beijing, preside...|[beijing, preside...|(262144,[789,1232...|
17290|After ‘The Bigges...|Danny Cahill stoo...|[danny, cahill, s...|[danny, cahill, s...|(262144,[571,991,...|
17291|First, a Mixtape....|Just how is Hil...|[just, how, is, h...|[hillary, kerr, f...|(262144,[991,1998...|
17292|Calling on Angels...|Angels are everyw...|[angels, are, eve...|[angels, everywhe...|(262144,[1712,236...|
17293|Weak Federal Powe...|With Donald J. Tr...|[with, donald, tr...|[donald, trump, t...|(262144,[14,632,2...|
17294|Can Carbon Captur...|THOMPSONS, Tex. ...|[thompsons, tex, ...|[thompsons, tex, ...|(262144,[14,329,5...|
17295|Mar-a-Lago, the F...|WEST PALM BEACH, ...|[west, palm, beac...|[west, palm, beac...|(262144,[408,1156...|
17296|How to form healt...|This article is p...|[this, article, i...|[article, part, s...|(262144,[14,2437,...|
17297|Turning Your Vaca...|It’s the season f...|[it, the, season,...|[season, family, ...|(262144,[15,571,1...|
17298|As Second Avenue ...|Finally. The Seco...|[finally, the, se...|[finally, second,...|(262144,[590,1232...|
17300|Dylann Roof Himse...| pages into the ...|[pages, into, the...|[pages, journal, ...|(262144,[14,619,8...|
17301|Modi’s Cash Ban B...|MUMBAI, India — ...|[mumbai, india, i...|[mumbai, india, b...|(262144,[14,1461,...|
17302|Suicide Bombing i...|BAGHDAD — A su...|[baghdad, suicide...|[baghdad, suicide...|(262144,[329,1728...|
17303|Fecal Pollution T...|SYDNEY, Australia...|[sydney, australi...|[sydney, australi...|(262144,[560,2317...|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [26]:
idf=IDF(inputCol='rawFeatures', outputCol='resultado')

In [27]:
idf_model=idf.fit(featurized_data)

Out[ 92 ]: 'inputCol: input column name. (current: rawFeatures)\nminDocFreq: minimum number of documents in which a term should appear for filtering (default: 0)\noutputCol: output column name. (default: IDF_0bda7527054b__output, current: resultado)'

In [28]:
rescaled_data=idf_model.transform(featurized_data)

In [29]:
rescaled_data.select('id','title','filtered','resultado').show()

+-----+--------------------+--------------------+--------------------+
 id| title| filtered| resultado|
+-----+--------------------+--------------------+--------------------+
17283|House Republicans...|[washington, cong...|(262144,[619,2326...|
17284|Rift Between Offi...|[bullet, shells, ...|(262144,[14,343,1...|
17285|Tyrus Wong, ‘Bamb...|[walt, disney, ba...|(262144,[14,535,5...|
17286|Among Deaths in 2...|[death, may, grea...|(262144,[75,535,7...|
17287|Kim Jong-un Says ...|[seoul, south, ko...|(262144,[2472,328...|
17288|Sick With a Cold,...|[london, queen, e...|(262144,[2564,538...|
17289|Taiwan’s Presiden...|[beijing, preside...|(262144,[789,1232...|
17290|After ‘The Bigges...|[danny, cahill, s...|(262144,[571,991,...|
17291|First, a Mixtape....|[hillary, kerr, f...|(262144,[991,1998...|
17292|Calling on Angels...|[angels, everywhe...|(262144,[1712,236...|
17293|Weak Federal Powe...|[donald, trump, t...|(262144,[14,632,2...|
17294|Can Carbon Captur...|[thompsons, tex, ...|(262144,[14,329,5...|
17295|Mar-a-Lago, the F...|[west, palm, beac...|(262144,[408,1156...|
17296|How to form healt...|[article, part, s...|(262144,[14,2437,...|
17297|Turning Your Vaca...|[season, family, ...|(262144,[15,571,1...|
17298|As Second Avenue ...|[finally, second,...|(262144,[590,1232...|
17300|Dylann Roof Himse...|[pages, journal, ...|(262144,[14,619,8...|
17301|Modi’s Cash Ban B...|[mumbai, india, b...|(262144,[14,1461,...|
17302|Suicide Bombing i...|[baghdad, suicide...|(262144,[329,1728...|
17303|Fecal Pollution T...|[sydney, australi...|(262144,[560,2317...|
+-----+--------------------+--------------------+--------------------+
only showing top 20 rows

In [30]:
keywordTF = HashingTF.transform([keyword.lower()])
keywordHashValue = int(keywordTF.indices[0])

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-1557488164861905> in <module> () 
 ----> 1 keywordTF = HashingTF . transform ( [ keyword . lower ( ) ] ) 
 2 keywordHashValue = int ( keywordTF . indices [ 0 ] ) 

 TypeError : transform() missing 1 required positional argument: 'dataset'

In [31]:
wordCountsDF = (sinsw
                .groupBy('filtered').count())
wordCountsDF.show()

+--------------------+-----+
 filtered|count|
+--------------------+-----+
[annapolis, md, e...| 1|
[trumpian, sense,...| 1|
[washington, scie...| 1|
[one, night, six,...| 1|
[glen, elder, kan...| 1|
[washington, pres...| 1|
[emmanuelle, riva...| 1|
[watching, male, ...| 1|
[want, get, brief...| 1|
[united, nations,...| 1|
[washington, cong...| 1|
[palatine, ill, t...| 1|
[garden, city, ma...| 1|
[washington, supr...| 1|
[los, angeles, go...| 1|
[beginning, proce...| 1|
[appeals, court, ...| 1|
[heads, 42nd, sea...| 1|
[norfolk, va, hug...| 1|
[ford, spend, 640...| 1|
+--------------------+-----+
only showing top 20 rows

In [32]:
ia= count_tokens('Trump')

Column<b'<lambda>(Trump)'>